In [1]:
import logging

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.DEBUG)

In [2]:
from experiments._3_gnn_models.gnn_social_RT_pred import *

Switching to API Credentials #0


In [3]:
config_name = "mid"

In [4]:
(
    N_EPOCHS,
    LIMIT_SAMPLE_USERS,
    LIMIT_SAMPLE_TWEETS,
    ONLY_ACTIVE_USERS,
    N_BATCHES_LOG_FREQUENCY,
    BATCH_SIZE
) = get_running_config(config_name)


        Config: mid
        ----------------------
        N_EPOCHS: 20,
        LIMIT_SAMPLE_USERS: 2,
        LIMIT_SAMPLE_TWEETS: None,
        ONLY_ACTIVE_USERS True,
        N_BATCHES_LOG_FREQUENCY: 1
        BATCH_SIZE: 1024
    


In [5]:
g = load_ig_graph()

In [6]:
centralities = load_precomputed_centralities()

In [7]:
train_samples, test_samples = get_samples(n_users=LIMIT_SAMPLE_USERS,
                                          n_tweets=LIMIT_SAMPLE_TWEETS,
                                          only_active_users=ONLY_ACTIVE_USERS)

print("train and test users:")
print(len(train_samples))
print(len(test_samples))


train and test users:
2
2


In [8]:
train_samples.keys()

dict_keys(['101047126', '110325813'])

In [9]:
user_id = "101047126"

In [10]:
len(train_samples[user_id]), len(test_samples[user_id])

(3500, 1500)

Check if an XGBoost on these users produces something reasonable

In [11]:
from experiments.datasets import extract_only_features

In [15]:
def get_Xy(user_id, tweet_ids):
    s = open_session()
    
    user = s.query(User).get(user_id)
    neighbours = get_level2_neighbours(user, s)
    # remove central user from neighbours
    neighbour_users = [u for u in neighbours if u.id != user.id]

    tweets = [s.query(Tweet).get(t) for t in tweet_ids]

    X = extract_only_features(tweets, neighbours)

    y = extract_target(tweets, user)
    
    return X, y

In [ ]:
X_tr, y_tr = get_Xy(user_id, train_samples[user_id])

/tmp/ipykernel_777933/2494785587.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user = s.query(User).get(user_id)
/tmp/ipykernel_777933/2494785587.py:9: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  tweets = [s.query(Tweet).get(t) for t in tweet_ids]
DEBUG:root:Starting
DEBUG:root:Processed 3.6 % of neighbours
DEBUG:root:Processed 7.5 % of neighbours
DEBUG:root:Processed 11.3 % of neighbours
DEBUG:root:Processed 15.2 % of neighbours
DEBUG:root:Processed 19.0 % of neighbours
DEBUG:root:Processed 22.8 % of neighbours
DEBUG:root:Processed 26.7 % 

In [ ]:
X_tr.shape

In [ ]:
y_tr.shape

In [ ]:
test_samples[user_id]

In [ ]:
X_te, y_te = get_Xy(user_id, test_samples[user_id])

In [ ]:
# train and evaluate


Are there many repeated rows?